# Modifiying Network Flow Algebra for Modelling Non-point Pollution for Interactive Use in Jupyter Notebook

Our project is related to the nitrogen modeling case study paper by Shook, Merson, and Wentz: "Network Flow Algebra for Modelling Non-Point Source Pollution: A Case Study in Niantic Watershed, Connecticut" (2018, in review). Their method of non-point pollution nitrogen modeling utilized network flow algebra to create a more efficient modeling techniqe that only counts the net flow path from the nitrogen source to sink. Their method of reach reduction allows the program to not have to calculate from the pollution source, to non-source, to sink. Our goal is to use a Jupyter Notebook to add widgets that allow users to modify the input data and receive a shapefile of flow paths with accumulated nitrogen flow attributed. In the original work, the nitrogen load values are set in the stream flow shapefile based on land use NLCD values. For our modification of the original code, the user will be able to use a sliding scale widget to decide how much nitrogen (or other pollutant, potentially) will be associated with land use types that are sources for non-point pollution.

To create a generalized, widely applicable workflow, the Jupyter Notebook uses two slider bars, each with a range of values representing a land use value of "Cultivated Crops" (30 - 50 N/acre/year) or "Developed Land" (40 - 60 N/acre/year). NLCD values of (22) Developed (Low Intensity), (23) Developed (Medium Intensity), and (82) Cultivated Crops are used to define Cultivated Crops and Developed Land. As the slider is changed for a given land use value, that value will be applied to the stream flow direction network; that is, in a given cell of the NLCD dataset, whatever segment of streamline passes through that cell will be attributed with the land use value. The flow direction network will then be passed into the existing code to calculate the output nitrogen load, creating a shapefile that includes the stream flow network attributed with nitrogen values.


This project took many forms over the course of the semester. Initially, we tried to create the goal of the project using a new study area, the Lake Superior-North watershed, HUC 04010101. We obtained data from the Minnesota Pollution Control Agency (MPCA) for surface water pollution by watershed based on the USGS HUD categorization to use as validation for Nitrogen surface water pollution. We also obtained a 30 m Land Use NLCD raster from Data.gov and clipped to our study watershed and stream flow direction for 30 m DEM from Data.gov and clipped to our watershed.

## INPUT FILES
These files will come to you in a zipped folder. Feel free to change the files paths to match where they are stored on your computer. 

In [ ]:
# This stores the filepath for the input NLCD raster for step one
landcover = "C:/Users/Lauren/Documents/Graduate Coursework/Fall 2018/GEOG 5543 Advanced Geocomputing/NLCD_Niantic.tif"

# The stream flow network shapefile to be rasterized in step two
flowdirect = "C:/Users/Lauren/Documents/Graduate Coursework/Fall 2018/GEOG 5543 Advanced Geocomputing/StreamLoad-20181029T212254Z-001/StreamLoad/InputPath_utmz18n.shp"

# Rasterized lines created in ArcGIS, would-be outcome of step two, used in step three 
lines = "C:/Users/Lauren/Documents/Graduate Coursework/Fall 2018/GEOG 5543 Advanced Geocomputing/flowline_ras.tif"

# Sample Niantic River flow path shapefile, used in step 4. 
flow_sample = 'C:/Users/Lauren/Documents/Graduate Coursework/Fall 2018/GEOG 5543 Advanced Geocomputing/Niantic_sample.shp'

## Step one: Set widget values for Land Use Types

In [ ]:
# Import modules
import ipywidgets as widgets
from IPython.display import display
from osgeo import gdal
from osgeo import ogr 
import rasterio
import pprint

In [1]:
# Create Jupyter Widget sliders for cultivated crops and developed land and display. 
w1 = widgets.IntSlider(description='CultivatedCrops', max=60, min=40)
w2 = widgets.IntSlider(description='DevelopedLand', max=50, min=30)

display(w1)
display(w2)

IntSlider(value=40, description='CultivatedCrops', max=60, min=40)

IntSlider(value=30, description='DevelopedLand', max=50, min=30)

In [2]:
# Create variables from slider values to pass into raster reclassification code. 
xw1 = w1.value
xw2 = w2.value

# Prints values set in sliders to make sure sliders are working right!
print(w1.value)
print(w2.value)

56
36


In [ ]:
# This variable is the output file path for the reclassified landcover raster that will be produced. 
# You can modify the file path to put it where you would like
landcover_reclass = "C:/Users/Lauren/Documents/Graduate Coursework/Fall 2018/GEOG 5543 Advanced Geocomputing/NLCD_Niantic_reclass.tif"

# Open the NLCD tif as an array
driver = gdal.GetDriverByName("GTiff")
file = gdal.Open(landcover)
band = file.GetRasterBand(1)
lista = band.ReadAsArray()

# Raster reclassification - set values of all cells that are not developed - low intensity, developed - medium intensity, or cultivated crop to 0.
# Set values of all cells that are developed land and cultivated crop to value determined by slider. 
for j in  range(file.RasterXSize):
    for i in  range(file.RasterYSize):
        if lista[i,j] == 22: # Use xw2 variable from w2 slider for values of developed - low intensity set by slider. 
            lista[i,j] = xw2
        elif lista[i,j] == 23: # Use xw2 variable from w2 slider for values of developed - medium intensity set by slider. 
            lista[i,j] = xw2
        elif lista[i,j] == 82: # Use xw1 variable from w1 slider for values of cultivated crop set by slider. 
            lista[i,j] = xw1
        else:
            lista[i,j] = 0

# Create reclassified land cover raster
outFile = driver.Create( landcover_reclass, file.RasterXSize , file.RasterYSize , 1)
outFile.GetRasterBand(1).WriteArray(lista)

# Define spatial reference based on input land cover raster
proj = file.GetProjection()
georef = file.GetGeoTransform()
outFile.SetProjection(proj)
outFile.SetGeoTransform(georef)
outFile.FlushCache()
print("Raster reclassified.")

## Step two: Rasterize Stream Flow Lines 

Rasterizing the stream flow lines shapefile did not become functional. We tried it both with the GDAL RasterizeLayer() function and the rasterize function from the command line. I think something is buggy with this function. I read a lot of recent issues with it online and I spent a whole lot of time trying to get it to work. Ultimately, due to time constraints, we decided to make the rasterized lines data in ArcGIS to use in other parts of the processs. 

I have left some commented out code of things that I was trying to use to debug and test if adding different things would work. When you run the Test to see if the layers are writing you can see target_ds and source_layer. The function is silently failing somewhere. The expected outcome of this block of cell is a zero and a tiff that has all zero values. One potential place we are missing is setting the projection in "Create the destination data source." I left a comment in where I was trying to do that, but we moved on to work on other aspects and ran out of time.  

In [ ]:
#Use GDAL to open the reclassified NCLD landcover Tiff to get the cell size information and GeoTransform information

#Opens landcover_reclass 
raster = gdal.Open(landcover_reclass)
gt =raster.GetGeoTransform()
print(gt)

#Prints the pixel size to be used in the new raster for the rasterized lines
pixelSizeX = gt[1]
pixelSizeY =-gt[5]
print("pixelSizeX:", pixelSizeX)
print("pixwlSizeY:", pixelSizeY)

In [ ]:
#Recipe from GDAL documentation to convert OGR File (e.g. Shapefile) to Raster

# Define pixel_size and NoData value of new raster
pixel_size = 30
NoData_value = -99999

# Filename of input OGR file
vector_fn = flowdirect

# Filename of the raster Tiff that will be created
raster_fn = 'lines_1.tif'

# Open the data source and read in the extent and spatial reference
source_ds = ogr.Open(vector_fn)
source_layer = source_ds.GetLayer()
x_min, x_max, y_min, y_max = source_layer.GetExtent()
spatialref = source_layer.GetSpatialRef()

# Create the destination data source
x_res = int((x_max - x_min) / pixel_size)
y_res = int((y_max - y_min) / pixel_size)
target_ds = gdal.GetDriverByName('GTiff').Create(raster_fn, x_res, y_res, 1, gdal.GDT_UInt32)
target_ds.SetGeoTransform((x_min, pixel_size, 0, y_max, 0, -pixel_size))
#target_ds.SetProjection(source_ds)
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(NoData_value)

#Test to see if the layers were writin
#print(target_ds, source_layer)

# Rasterize function
gdal.RasterizeLayer(target_ds, [1], source_layer, attributes=["HYDROID"])


band.FlushCache()

## Step Three:  Create a dictionary using HYDROID of flow path raster as KEY and cumulative Nitrogen Load value from reclassified NLCD raster as VALUE. 

For this section the data used is the reclassified NLCD raster (dataset) and the rasterized lines (dataset_lines). This step is the least developed of our methods both because it is a complicated problem and because of time constraints. We did not get to this step until a few days before the due date and ran into issues with data. The process that I wanted to finish for this loop would be to loop through the loads_array and write the HYDROID to a dictionary as the key from the lines_array if it is not in the dictionary, add the load to each hydroid so that the value is the cumuluative nitrogen for each HYDROID. 

Honestly, this section felt like a whole project in itself and starting it a few days before the due date felt very overwhelming knowing that I would not have enough time. I've left in all of the places that I tested printing trying to make sure I was seeing the correct values because I had some set backs with looping correctly and with the input data. If I had been able to start this a month earlier I know I would have gotten a lot farther. 

In [ ]:
#Open the NLCD with loads assigned based on widgets using rasterio
dataset = rasterio.open(landcover_reclass)

#Reads band1 as an array (we only have one band)
band1 = dataset.read(1)
print(band1)

In [ ]:
#Open rasterized lines tif with rasterio
dataset_lines = rasterio.open(lines)

#Reads band1 as an array
band1_lines = dataset_lines.read(1)
print(band1_lines)

In [ ]:
#Variables to refer to the opened tifs of the loads and the lines
loads_array = band1
lines_array = band1_lines

#Dictionary to store key and value pairs.
load_count = {} 


#print(loads_array)
for r in range(len(loads_array)):
    for c in range(len(loads_array[r])):
        #print(loads_array[r])
        
        #I know we will need to set r to zero within the loop, but I'm not sure about this placement
        r == 0
        c == 0
        
        #I know this part errors out, I was trying to figure out how to read the lines_array while inside the loop
        obj_id = lines_array[r][c]
        
        #print(loads_array[r][c]) 
        if loads_array[r][c]>0:
            load = loads_array[r][c]
        
        #In the process of trying to understand how to update the dictionary 
        if obj_id not in load_count:
            load_count.update({obj_id: 0})
            
        #Had not yet gotten to the point of adding the loads by HYDROID

## Step Four: Create new field (NLoad) in original flow path shapefile and append the calculated nitrogen load value based on the HYDROID. 

In this section, we create a sample dictionary from the Niantic River flow path shapefile with HYDROID as the key and a made-up nitrogen load as the value. We then use ogr to open the shapefile, append a field called NLoad, and populate the field with the values in the dictionary based on their corresponding HYDROID. Because we were not able to successfully complete prior steps, this seemed like the best way to get as far as we possibly could on the project. 

In [3]:
# Sample dictionary with keys as HYDROID and value as Nload 
NDict = {'569747':0,'569748':50,'569749':0,'569750':100,'569833':30,'569834':0,'569835':0,'570105':50} 

for key in NDict:
    print("HYDROID", key, "has N load of", NDict[key])

HYDROID 569747 has N load of 0
HYDROID 569748 has N load of 50
HYDROID 569749 has N load of 0
HYDROID 569750 has N load of 100
HYDROID 569833 has N load of 30
HYDROID 569834 has N load of 0
HYDROID 569835 has N load of 0
HYDROID 570105 has N load of 50


In [ ]:
# This appends attribute field NLoad to Niantic_sample.shp

# Open Niantic_sample.shp and get field names
source = ogr.Open(flow_sample, update=True)
layer = source.GetLayer()
layer_defn = layer.GetLayerDefn()
field_names = [layer_defn.GetFieldDefn(i).GetName() for i in range(layer_defn.GetFieldCount())]

# Add a new field (NLoad) to existing shapefile (Niantic_sample.shp) 
new_field = ogr.FieldDefn('NLoad', ogr.OFTReal)
layer.CreateField(new_field)
print (len(field_names), 'NLoad' in field_names)
print("NLoad field added to shapefile!")

# Save and close the file
source = None 

In [ ]:
# This updates NLoad field in Niantic_sample.shp with NLoad values from dictionary NDict. 

# Set variables so using ogr commands is easier. 
driver = ogr.GetDriverByName('ESRI Shapefile')
Source = driver.Open(flow_sample, 1) # Uses variable set above to access shapefile. 1 sets overwrite to True.

layer = Source.GetLayer()
feature = layer.GetNextFeature() # Tells ogr to keep looking until there are no more features in the file.

# Loop through features in the shapefile. 
while feature:
    hydroID = feature.GetField('HYDROID') # Look at HYDROID field in shapefile
    load = NDict[str(hydroID)] # For each HYDROID, cast the load as an integer to agree with field type.
    feature.SetField("NLoad", int(load)) # Set the field to equal the value associated with HYDROID
    layer.SetFeature(feature) # Save the value to the shapefile
    feature = layer.GetNextFeature() # Advance through all features in shapefile. 
    print(hydroID, load)

# Save and close the file 
source = None 